# 计算机视觉纳米学位项目

## 实战项目：图像标注

---

在这个notebook中，你将要使用已被训练的模型为测试数据集中的图像生成标注。

我们将对该notebook**进行评分**。

你可以通过点击以下链接导航到该notebook：
- [Step 1](#step1): 获取测试数据集的数据加载器
- [Step 2](#step2): 加载训练模型
- [Step 3](#step3): 完成取样器
- [Step 4](#step4): 清理标注
- [Step 5](#step5): 生成预测！

<a id='step1'></a>
## Step 1: 获取测试数据集的数据加载器

在运行下面的代码单元格之前，请在`transform_test`中定义要用于预处理测试图像的转换。

请确保你在此处定义的转换与你在**2_Training.ipynb**中用于预处理训练图像的转换是一致的。例如，如果你对训练图像进行了归一化，则还应对测试图像应用相同的归一化处理。

In [ ]:
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from torchvision import transforms

# TODO #1: Define a transform to pre-process the testing images.
transform_test = ...

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Create the data loader.
data_loader = get_loader(transform=transform_test,    
                         mode='test')

在应用预处理之前，运行下面的代码单元格，将示例测试图像可视化。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Obtain sample image before and after pre-processing.
orig_image, image = next(iter(data_loader))

# Visualize sample image, before pre-processing.
plt.imshow(np.squeeze(orig_image))
plt.title('example image')
plt.show()

<a id='step2'></a>
## Step 2: 加载训练模型

在下一个代码单元格中，我们定义了一个`device`，你将使用它将PyTorch张量移动到GPU中（如果CUDA可用的话）。在继续下一步之前，请运行此代码单元格。

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

在运行下面的代码单元格之前，请完成以下任务。

### 任务 #1

在下一个代码单元格中，你将从之前的notebook（即**2_Training.ipynb**）中加载已被训练的编码器和解码器。要实现此目的，你必须在`models/`文件夹中定义已保存的编码器和解码器文件的名称。例如，如果你使用5个epoch训练了该模型并在每个epoch后保存了权重，则这些名称应为 `encoder-5.pkl`和`decoder-5.pkl`。

### 任务＃2

插入嵌入尺寸和解码器隐藏层的尺寸，与`decoder_file`中选定的pickle文件相对应。

In [ ]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

import os
import torch
from model import EncoderCNN, DecoderRNN

# TODO #2: Specify the saved models to load.
encoder_file = ... 
decoder_file = ...

# TODO #3: Select appropriate values for the Python variables below.
embed_size = ...
hidden_size = ...

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.eval()

# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

<a id='step3'></a>
## Step 3: 完成取样器

在执行下一个代码单元格之前，必须要编写**model.py**中的`DecoderRNN`类的`sample` 方法。该方法应接收包含对应于单个图像的嵌入输入特征的PyTorch张量`features`作为输入。

作为输出，它应该返回一个Python列表`output`，用于指示预测的语句。 `output[i]`是一个非负整数，用于标识句子中预测的第`i`个标记。你可以通过检查`data_loader.dataset.vocab.word2idx` 或 `data_loader.dataset.vocab.idx2word`来探索整数和标记之间的对应关系。

实现`sample`方法后，运行下面的代码单元格。如果单元格返回一个断言错误，请在继续操作之前按照说明修改代码，但请勿修改下面单元格中的代码。

In [ ]:
# Move image Pytorch Tensor to GPU if CUDA is available.
image = image.to(device)

# Obtain the embedded image features.
features = encoder(image).unsqueeze(1)

# Pass the embedded image features through the model to get a predicted caption.
output = decoder.sample(features)
print('example output:', output)

assert (type(output)==list), "Output needs to be a Python list" 
assert all([type(x)==int for x in output]), "Output should be a list of integers." 
assert all([x in data_loader.dataset.vocab.idx2word for x in output]), "Each entry in the output needs to correspond to an integer that indicates a token in the vocabulary."

<a id='step4'></a>
## Step 4: 清理标注

在下面的代码单元格中，完成`clean_sentence`函数。它应把一个整数列表（对应于**Step 3**中的变量`output`）作为输入并返回相应的预测语句（作为单个Python字符串）。

In [ ]:
# TODO #4: Complete the function.
def clean_sentence(output):
    return sentence

完成上面的`clean_sentence`函数后，运行下面的代码单元格。如果单元格返回一个断言错误，请在继续操作之前按照说明修改代码。

In [ ]:
sentence = clean_sentence(output)
print('example sentence:', sentence)

assert type(sentence)==str, 'Sentence needs to be a Python string!'

<a id='step5'></a>
## Step 5: 生成预测！

在下面的代码单元格中，我们编写了一个函数(`get_prediction`，你可以使用该函数遍历测试数据集中的图像并输出模型的预测描述。

In [ ]:
def get_prediction():
    orig_image, image = next(iter(data_loader))
    plt.imshow(np.squeeze(orig_image))
    plt.title('Sample Image')
    plt.show()
    image = image.to(device)
    features = encoder(image).unsqueeze(1)
    output = decoder.sample(features)    
    sentence = clean_sentence(output)
    print(sentence)

运行下面的代码单元格，测试此函数的运行方式。如果你愿意的话，可以尝试多次运行。

In [ ]:
get_prediction()

作为此项目的最后一项任务，你要做的是遍历图像，直到找到四个感兴趣的图像标注对：
- 其中，有两个应该包括图像标注对，且该标注对会突出显示模型表现良好的实例。
- 另外两个应突出显示图像标注对，且该标注对会突出显示模型效果不佳的实例。

使用下面的四个代码单元格来完成此任务。

### 该模型表现良好！

使用接下来的两个代码单元格来遍历所有标注。如果遇到两张图像有相对准确的标注，请保存该notebook。

In [ ]:
get_prediction()

In [ ]:
get_prediction()

### 该模型表现不佳......

使用接下来的两个代码单元格来遍历所有标注。如果遇到两个图像有相对不准确的标注，请保存该notebook。

In [ ]:
get_prediction()

In [ ]:
get_prediction()